In [ ]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.insert(0, "../")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skorch import NeuralNetClassifier

import utils.loader as ld
import utils.image_processing as ip
%aimport models.raph_model 
%aimport utils.metrics

In [ ]:
tr_X, tr_y, tr_classes, te_X, te_y, te_classes = ld.load()

In [ ]:
ip.imshow(tr_X[0, 0, :, :])

In [ ]:
tr_X = tr_X.view(-1, 1, 14, 14).squeeze()
tr_classes = tr_classes.view(-1, 1).squeeze()
te_X = te_X.view(-1, 1, 14, 14).squeeze()
te_classes = te_classes.view(-1, 1).squeeze()

In [ ]:
for i in range(1000):
    
tr_X.shape

In [ ]:
class LeNet_5(nn.Module):
    """ nn.Linear(84, 10)(
        nn.Linear(120, 84)(
        nn.Linear(16 * 5 * 5, 120)(
            nn.Conv2d(6, 16, 5)(
                nn.MaxPool2d(2, 2)(
                    nn.Conv2d(2, 6, (5, 5))(tr_X))).view(-1, 16*5*5)))) """  
                      
    def __init__(self):
        super(LeNet_5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (1, 1))
        self.pool = nn.MaxPool2d((2, 2), stride=1)
        self.conv2 = nn.Conv2d(6, 16, (1, 1))
        self.conv3 = nn.Conv2d(16, 120, (1, 1))
        self.fc1 = nn.Linear(17280, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1) # !!!!
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(2, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x)

In [ ]:
def train(model, optimizer, criterion, epoch, tr_X, tr_y):
    model.train()
    for bc, (input_, target) in enumerate(zip(tr_X.split(batch_size), tr_y.split(batch_size))):
        optimizer.zero_grad()
        output = model(input_)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if bc % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, bc * len(input_), len(tr_X),
                100. * bc / len(tr_X), loss.item()))

In [ ]:
def test (model, criterion, te_X, te_y):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in zip(te_X.split(batch_size), te_y.split(batch_size)):
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(tr_X)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(tr_X),
        100. * correct / len(tr_X)))

In [ ]:
model(tr_X[:64, :, :])

In [ ]:
lr, nb_epochs, batch_size = 1e-1, 10, 64
model = LeNet_5()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()


mu, std = tr_X.mean(), tr_X.std()
tr_X.sub_(mu).div_(std)

mu, std = te_X.mean(), te_X.std()
te_X = te_X.sub_(mu).div_(std)

In [ ]:
for epoch in range(1, nb_epochs):
        train(model, optimizer, criterion, epoch, tr_X, tr_y)
        test(model, criterion, te_X, te_y)